In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# !pip3 install torch
# !pip3 install torchvision

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim
from torch.autograd import Variable

In [0]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])

In [0]:
trainset = datasets.CIFAR10('./data', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50, shuffle=True)
# trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Files already downloaded and verified


In [0]:
testset = datasets.CIFAR10('./data', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=50, shuffle=True)

# testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Files already downloaded and verified
torch.Size([50, 3, 32, 32])


In [0]:
for data, labels in testloader:
  print(data.shape)
  break

torch.Size([50, 3, 32, 32])


In [0]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 8 * 8, 10) 

        self.softmax = torch.nn.Softmax(dim = 1)
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 8, 8)
        # out.size(0): 100
        # New out size: (100, 32*8*8)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        out = self.softmax(out)
        return out

In [0]:
# Create an object of class MLP
model = CNNModel()

# Define the loss
criterion = torch.nn.CrossEntropyLoss()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr = 0.05)

In [0]:
epochs = 100
for e in range(epochs):
    print("Epoch ", (e + 1))
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        logits = model.forward(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        accuracy = 0
        with torch.no_grad():
            for images, labels in testloader:
                logit = model(images)
                ps = logit#torch.exp(logit)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        print("Loss on training set: ", running_loss)
        print("Accuracy on test set: ", end = " ")
        print((accuracy / len(testloader)).data.numpy())

Epoch  1
Loss on training set:  2190.154004096985
Accuracy on test set:  0.3469
Epoch  2
Loss on training set:  2066.225322008133
Accuracy on test set:  0.43839988
Epoch  3
Loss on training set:  2015.6646366119385
Accuracy on test set:  0.4706999
Epoch  4
Loss on training set:  1979.7148439884186
Accuracy on test set:  0.4897999
Epoch  5
Loss on training set:  1947.0443089008331
Accuracy on test set:  0.5097
Epoch  6
Loss on training set:  1922.5798836946487
Accuracy on test set:  0.54429984
Epoch  7
Loss on training set:  1900.2027895450592
Accuracy on test set:  0.5584
Epoch  8
Loss on training set:  1880.6006295681
Accuracy on test set:  0.5656002
Epoch  9
Loss on training set:  1863.1601527929306
Accuracy on test set:  0.58640015
Epoch  10
Loss on training set:  1846.6627342700958
Accuracy on test set:  0.59710014
Epoch  11
Loss on training set:  1832.8818269968033
Accuracy on test set:  0.60230017
Epoch  12
Loss on training set:  1820.5944964885712
Accuracy on test set:  0.6082
E

** Choose any dataset available online for multi class classification(minimum 3 classes) and try and train a CNN on that dataset.
** You should try and write your own dataloader function and don't use pytorch's inbuilt one.
** Experiment with the number of layers, kernel size, strides, padding.
** Get the best accuracy you can get.
** Don't use the datasets which are inbuilt in pytorch.